# Compute 4He form factors from 1-body densities

## Init

In [ ]:
from typing import Optional
import os
import re
import pandas as pd
import numpy as np
import sympy
from scipy.interpolate import interp1d

import matplotlib.pylab as plt
import seaborn as sns

import gvar as gv
import lsqfit

## Utility Functions

In [ ]:
def parse_label(fit, subs=True):
    """Parses the fit function and posteriror to latex label.
    
    If subs = False, don't substitude in posterior parameters.
    """
    expressions = {}
    values = {}
    for key, val in fit.p.items():
        if hasattr(val, "__iter__"):
            expr = sympy.symbols(" ".join([f"{key}{n}" for n, el in enumerate(val)]))
        else:
            expr = sympy.Symbol(key)

        expressions[key] = expr

        if hasattr(expr, "__iter__"):
            for ee, vv in zip(expr, val):
                values[sympy.latex(ee)] = vv
        else:
            values[sympy.latex(expr)] = val

    f_expr = fit.fcn(
        x=sympy.Symbol("x"), p={key: expr for key, expr in expressions.items()}
    )

    s = sympy.latex(f_expr)
    if subs:
        for pat, sub in values.items():
            s = s.replace(pat, str(sub))
            s = re.sub(r"e\+?([\-]?)[0]*([0-9]+)", " 10^{\g<1>\g<2>}", s)
    return re.sub(r"\+\s+\-", "-", s)

In [ ]:
COLORS = sns.color_palette("hls", 8)


def plot_fit(
    fit: lsqfit.nonlinear_fit,
    ax: Optional[plt.Axes] = None,
    color=None,
    label=None,
    plot_data: bool = True,
) -> plt.Axes:
    """Plots a nonlinear_fit (data and fit result)."""
    _, ax = plt.subplots(figsize=(3, 2), dpi=300) if not ax else (None, ax)

    y_mean, y_sdev = gv.mean(fit.data[1]), gv.sdev(fit.data[1])
    if plot_data:
        ax.errorbar(
            fit.data[0],
            y_mean,
            y_sdev,
            marker="o",
            ms=2,
            ls="None",
            capsize=2,
            elinewidth=1,
            label="data",
            color=COLORS[0],
        )

    x_int = np.linspace(fit.data[0].min(), fit.data[0].max())
    y_fit = fit.fcn(x_int, fit.p)
    y_mean, y_sdev = gv.mean(y_fit), gv.sdev(y_fit)

    try:
        llabel = parse_label(fit)
        llabel = f"$f(x) = {label}$"
    except Exception as e:
        print(e)
        llabel = str(fit.p)

    ax.fill_between(
        x_int,
        y_mean - y_sdev,
        y_mean + y_sdev,
        color=color or COLORS[1],
        alpha=0.5,
        label=label or "fit",
    )

    ax.legend(
        fancybox=False, frameon=False, bbox_to_anchor=(1.1, 0.5), loc="center left"
    )

    return ax

## Definitions (needs user input)

In [ ]:
HBARC = 197.3269804
MN = 938.918

For this code to run, you have to specify two directories:
    
1. `DATA_4HE_AV18`  which contains the `compton-dens-4he-av18-...-rho1b.dat`  files
2. `DATA_FF` which contains the single proton & neutron form factor data as specified in [[1707.09063]](https://arxiv.org/abs/1707.09063 )

In [ ]:
DATA = "/home/ckoerber/data"
DATA_4HE_CHSMS = "/home/ckoerber/data/nuc/4he/chsms"

DENSITIES = [
    f for f in os.listdir(DATA_4HE_CHSMS) if f.endswith(".dat") and "th=1.80E+02" in f
]
print("First rho1b file", DENSITIES[0])

DATA_FF = "/home/ckoerber/data/nuc/NucleonFFData"
print("\nFF data:", os.listdir(DATA_FF))

## Parse file names to variables

In [ ]:
patterns = (
    r"compton-dens-(?P<nuc>[0-9A-z]+)",
    r"\-(?P<potential>[a-z0-9\+]+)",
    r"\-(?P<order>[a-z0-9\+]+)",
    r"\-cut=(?P<cut>[0-9]+)",
    r"\-(?P<empot>[a-zA-Z]+)",
    r"\-(?P<cmpi>(?:[a-z0-9]+))",
    r"\-Lam=(?P<Lam>(?:[\.0-9]+))",
    r"\-c1=(?P<c1>(?:[\-\.0-9]+))",
    r"\-c3=(?P<c3>(?:[\-\.0-9]+))",
    r"\-c4=(?P<c4>(?:[\-\.0-9]+))",
    r"\-Lamnum=(?P<lambda>(?:[0-9\.e\+]+))",
    r"\-tnfcut=(?P<tnfcut>(?:[0-9]+))",
    r"\-om=(?P<omega>(?:[0-9\.]+E[\+\-][0-9]+))",
    r"\-th=(?P<theta>(?:[0-9\.E\+]+))",
    r"\-nx=(?P<nx>(?:[0-9]+))",
    r"\-nphi=(?P<nphi>(?:[0-9]+))",
    r"\-np12\=np34\=(?P<np12_np34>(?:[0-9\+]+))",
    r"\-np3\=(?P<np3>(?:[0-9\+]+))",
    r"\-nq4\=nq=(?P<nq4_nq>(?:[0-9\+]+))",
    r"\-j12max=(?P<j12max>(?:[0-9]+))",
    r"\-lmax=(?P<lmax>(?:[0-9]+))",
    r"\-lsummax=(?P<lsummax>(?:[0-9]+))",
    r"\-tau4max=(?P<tau4max>(?:[0-9]+))",
    r"\-rho1b\.dat",
)
ppat = ""
data = None
for pat in patterns:
    ppat += pat
    match = re.search(ppat, DENSITIES[0])
    if not match:
        print(data)
        raise ValueError(ppat)
    else:
        data = match.groupdict()
pattern = re.compile("".join(patterns))

In [ ]:
dtypes = {
    int: ["nx", "nphi", "j12max", "lmax", "lsummax", "tau4max"],
    float: ["lambda", "omega", "theta"],
}

In [ ]:
data = [pattern.search(f).groupdict() for f in DENSITIES]
df = pd.DataFrame(data)

print(df.describe().T)

for dtype, cols in dtypes.items():
    for col in cols:
        df[col] = df[col].astype(dtype)

df["file"] = DENSITIES
df = df.sort_values("omega").reset_index(drop=True)
df["q"] = df["omega"] / HBARC * 2

df.head(5)

Only varying quantities are omega and theta

## File parsing

The routines below parse the `rho1b` files.

Results are dictionaries with keys specifying the matrix elements and channels

In [ ]:
pp = r"MAXRHO1BINDEX\s+\=\s+(?P<max_rho_index>[0-9]+)"
pp += r".*"
pp += r"RHO1BINDX\s+\=(?P<rho_index>[0-9\*\,\-\s]+)"
pp += r".*"
pp += r"\/\s+(?P<om_theta>[0-9\.\-\+E ]+\n)"
pp += r"\s+(?P<rho>[0-9\.\-\+E\s]+\n)"


def parse_fortran_funny(string):
    """Converts fortran format arrys in file to python objects"""
    for pat, subs in {
        f"{key}*{val}": ", ".join([val] * int(key))
        for key, val in set(
            re.findall(r"([0-9]+)\*([\-0-9]+)", re.sub(r"\s+", " ", string))
        )
    }.items():
        string = string.replace(pat, subs)

    arr = np.array(list(map(int, string.split(","))))
    nd = len(arr) // 8
    return pd.DataFrame(
        data=arr.reshape([nd, 8]),
        columns=[
            "ms3_x2",
            "mt3_x2",
            "mjtot_x2",
            "ms3p_x2",
            "mt3p_x2",
            "mjtotp_x2",
            "k",
            "bk",
        ],
    )


parse = {
    "max_rho_index": int,
    "om_theta": lambda el: np.array([float(ee) for ee in el.split(" ") if ee]),
    "rho": lambda el: np.array([float(ee) for ee in el.split(" ") if ee]),
    "rho_index": parse_fortran_funny,
}


def parse_1bd(address):
    """Reads in one-body density files"""
    with open(address, "r") as inp:
        t = inp.read()
    dd = re.search(pp, t, re.MULTILINE | re.DOTALL).groupdict()
    for key, val in parse.items():
        dd[key] = val(dd[key])

    channels = dd["rho_index"]
    channels["rho"] = dd["rho"]
    dd["channels"] = channels
    return dd

In [ ]:
f = df.query("omega == 0").file.values[0]
dens = parse_1bd(os.path.join(DATA_4HE_CHSMS, f))

print("Dictionary keys are:", dens.keys())

In [ ]:
dens["channels"].sort_values("rho", ascending=False).head(10)

The entries `k` and `bk` encode angular dependence of the one-body operator
$$
O_{1} ( \vec k_{1} \  \mu' \mu \   \nu  \  \vec k ' \vec k ) \equiv \sum_{K=0}^{1}\sum_{\kappa=-K}^{K} \sqrt{\frac{4\pi}{2K+1}} \, k_{1}^{K} \, Y_{K\kappa}(\hat k_{1})
\tilde O_{1} ( K \kappa \  \mu' \mu \   \nu  \  \vec k ' \vec k )
$$
This would suggest, for the identity operation, $k = b_k = 0$.

*Question:*
Is this the same $k_1$ which appears in the SO force? If so, I don't have to evaluate 2b ops to get the SO done.

In [ ]:
IDX_QUERY = (
    "k == bk == 0"
    " and mt3_x2 == mt3p_x2"
    " and mjtot_x2 == mjtotp_x2"
    " and ms3_x2 == ms3p_x2"
)

norm = dens["channels"].query(IDX_QUERY)["rho"].sum()
print("norm (k==0==bk):", norm)
print("other contributions:", dens["channels"]["rho"].sum() - norm)

## File functions

In [ ]:
def compute_norm(data, ms3_x2=None):
    """Computes the norm of 1-body density given density data"""
    qquery = IDX_QUERY + " and ms3_x2 == @ms3_x2" if ms3_x2 is not None else IDX_QUERY
    return data["channels"].query(qquery)["rho"].sum()


def compute_norm_from_file(ff, data=DATA_4HE_CHSMS, ms3_x2=None):
    """Computes the norm of 1-body density given density file"""
    dd = parse_1bd(os.path.join(data, ff))
    return compute_norm(dd, ms3_x2=ms3_x2)


print("norm:", compute_norm(dens))
print("norm-p:", compute_norm(dens, ms3_x2=1))
print("norm-n:", compute_norm(dens, ms3_x2=-1))

## Read in GE data

In [ ]:
def dipole_FF(q2):
    """Returns dipole form factor named GD in notes.
    
    Here, q2 is in GeV**2"""
    return 1 / (1 + q2 / (0.71)) ** 2

In [ ]:
# Read in data
gp_data = (
    pd.read_csv(
        os.path.join(DATA_FF, "proton_baseline_sep272019_RE8414.dat"), sep="\s+",
    )
    .set_index("Q2")
    .rename(columns={"GEp/GD": "gep", "GMp/muGD": "gmp"})[["gep", "gmp"]]
)

# Mutliply by GD
gp_data["gep"] *= dipole_FF(gp_data.index.values)
gp_data["gmp"] *= dipole_FF(gp_data.index.values)

# convert q2 to MeV**2
gp_data.index = gp_data.index * 1000 ** 2

# Interpolate
gep = interp1d(gp_data.index, gp_data.gep.values, kind="cubic")
gmp = interp1d(gp_data.index, gp_data.gmp.values, kind="cubic")

In [ ]:
# Read in data
gn_data = (
    pd.read_csv(os.path.join(DATA_FF, "Ye2017gyb_neutron_lookup.dat"), sep="\s+",)
    .set_index("Q2")
    .rename(columns={"GEn/GD": "gen", "GMn/muGD": "gmn"})[["gen", "gmn"]]
)

# Mutliply by GD
gn_data["gen"] *= dipole_FF(gp_data.index.values)
gn_data["gmn"] *= dipole_FF(gp_data.index.values)

# convert q2 to MeV**2
gn_data.index = gn_data.index * 1000 ** 2

# Interpolate
gen = interp1d(gn_data.index, gn_data.gen.values, kind="cubic")
gmn = interp1d(gn_data.index, gn_data.gmn.values, kind="cubic")

In [ ]:
ax = gp_data.plot(y="gep")
gp_data.plot(y="gmp", ax=ax)
gn_data.plot(y="gen", ax=ax)
gn_data.plot(y="gmn", ax=ax)

ax.set_yscale("log")
ax.set_xscale("log")
ax.legend()
sns.despine()
plt.show()

In [ ]:
def ge_correction(k2, rho, mt3_x2):
    """Computes - k^2/8/MN * GE(k^2)
    
    k^2 is in MeV^2
    """
    gep_k2, gen_k2 = gep(k2), gen(k2)
    #return (-k2 / 8 / MN ** 2 * np.where(mt3_x2 == 1, gep_k2, gen_k2) * rho).sum()
    return (-k2 / 8 / MN ** 2 * rho).sum()


def compute_ge_correction_norm_from_file(ff, data=DATA_4HE_CHSMS):
    """Computes the GE correction given a density file
    """
    dd = parse_1bd(os.path.join(data, ff))
    tmp = dd["channels"].query(IDX_QUERY)
    rho = tmp["rho"].values
    mt3_x2 = tmp["mt3_x2"].values
    q2 = (dd["om_theta"][0] * HBARC * 2) ** 2
    if q2 < gp_data.index.min():
        q2 = gp_data.index.min()  # Avoid FF interpolation range issues
    return ge_correction(q2, rho, mt3_x2)

## Run computations

In [ ]:
df["norm"] = df.file.apply(compute_norm_from_file)
df["norm_p"] = df.file.apply(compute_norm_from_file, ms3_x2=1)
df["k2_correction"] = df.file.apply(compute_ge_correction_norm_from_file)
df.drop(columns=["lambda", "tnfcut", "file"]).sort_values("omega").head(10)

## Prepare plots & fits

In [ ]:
qquery = {"theta": 180, "nuc": "'4he'", "potential": "'chsms'", "Lam": "'550.000'"}
ddf = df.query(" and ".join(f"{key} == {val}" for key, val in qquery.items()))[
    ["q", "omega", "norm", "norm_p", "k2_correction"]
].sort_values("omega")
ddf = ddf.set_index("q")
print(ddf[["norm", "k2_correction"]])

### Run fits

In [ ]:
class FitArgs:
    def __init__(self, x, y, n_poly):
        self.n_poly = n_poly
        self.x = x
        self.y = y

    @staticmethod
    def poly_x2(x, p):
        """Computes $f(x) = 1 - \sum_{n=0} c_n x^{2n + 2}$.

        The prior p must contain the array c.
        """
        res = 1
        if hasattr(p["c"], "__iter__"):
            for n, cn in enumerate(p["c"]):
                res += x ** (2 * n + 2) * cn
        else:
            res += x ** (2) * p["c"]
        return res

    @property
    def prior(self):
        return {"c": gv.gvar(-np.ones(self.n_poly), 1 + np.arange(self.n_poly))}

    def __call__(self, z):
        dy = self.y * z
        newy = gv.gvar(self.y, dy)
        return dict(data=(self.x, newy), fcn=self.poly_x2, prior=self.prior)

In [ ]:
tmp = ddf.loc[:0.2]
x = tmp.index.values
y = tmp.norm.values / tmp.norm.values[0]


get_fit_args = FitArgs(x, y, n_poly=2)

# Compute posterior
fit, dy = lsqfit.empbayes_fit(1.0e-2, get_fit_args)

print("Nucleus")
print(qquery)

print("Numerical error of data (emperical bayes):", dy)

# Plot
fig, ax = plt.subplots(dpi=300, figsize=(3, 2))

for n_poly in range(1, 5):
    get_fit_args.n_poly = n_poly
    fit = lsqfit.nonlinear_fit(**get_fit_args(dy))

    print("\n fit function:", parse_label(fit, subs=False), "\n")
    print(fit.format(maxline=None))
    print("r0 = sqrt(-6*c1) * hbarc :", gv.sqrt(-fit.p["c"][0] * 6), "fm")

    plot_fit(
        fit,
        ax=ax,
        label=fr"$\chi^2 = {fit.chi2/fit.dof:1.1f}$ | $f(x) = {parse_label(fit, subs=True)}$",
        plot_data=n_poly == 1,
        color=COLORS[n_poly + 1],
    )

ax.set_ylabel("$F(q^2)$")
ax.set_title(
    ", ".join([f"{k}={v}" for k, v in qquery.items()]) + f", $\Delta y = {dy:1.1e}$",
    size=4,
)
ax.legend(loc="best", fontsize=4, frameon=False)

ax.set_xlabel("$q$ [fm$^{-1}$]")
sns.despine()
plt.show()

In [ ]:
fig.savefig("multi-poly-fit.pdf", bbox_inches="tight")

## Spin Orbit

This implements

$$ \frac{i}{m_N^2} \vec{\sigma}_1 \cdot ( \vec q \times \vec k_1) $$

where $k_1 = (\vec p_{1i} + \vec p_{1o})/2$ (in and out going first nucleon momentum) and $\vec q$ the external current momenutm.

Using momentum conservation $N(\vec p_{1i}) + J(\vec q) \to N(\vec p_{1o})$, this becomes
$$
\frac{i}{m_N^2} \vec{\sigma}_1 \cdot ( \vec q \times \vec p_{1i}) 
\to
\frac{i}{m_N^2} \sum_{K=1}^1 \sum_{\kappa=+1, -1} \sqrt{\frac{4 \pi}{2K + 1}} p_{1i} Y_{K \kappa}(\vec p_{1i})
O_{K \kappa m_{so}, m_{si}, m_t}(\vec q)
$$
with
$$
   O_{1 \kappa m_{so}, m_{si}, m_t}(\vec q)
   =
   \frac{\sqrt{2} q}{m_N^2}
$$
for $\vec q = q \vec e_z$ if $m_t$ and $m_j$ is conserved and $m_{si} \neq m_{so} = \kappa$.

In [ ]:
def spin_orbit(row, q):
    """in ifm**2
    
    density normalization?
    """
    mN = MN / HBARC
    fact = np.sqrt(2) * q / mN**2
    out = 0
    if (
        row["mt3_x2"] == row["mt3p_x2"]
        and row["mjtot_x2"] == row["mjtotp_x2"]
        and row["bk"] == 1
        and abs(row["k"]) == 1
        and row["ms3_x2"] == -row["ms3p_x2"]
        and row["ms3_x2"] == row["k"]
    ):
        out = fact * row["rho"]

    return out


def compute_so_from_file(ff, data=DATA_4HE_CHSMS):
    """Computes the GE correction given a density file
    """
    dd = parse_1bd(os.path.join(data, ff))
    qval = dd["om_theta"][2] # in ifm
    return dd["channels"].apply(spin_orbit, axis=1, args=(qval,)).sum()

In [ ]:
df["spin_orbit"] = df.file.apply(compute_so_from_file)
qquery = {"theta": 180, "nuc": "'4he'", "potential": "'chsms'", "Lam": "'550.000'"}
ddf = df.query(" and ".join(f"{key} == {val}" for key, val in qquery.items()))[
    ["q", "omega", "norm", "norm_p", "k2_correction", "spin_orbit"]
].sort_values("omega")
ddf = ddf.set_index("q")
ddf[["norm", "k2_correction", "spin_orbit"]]

In [ ]:
fig, ax = plt.subplots(dpi=300, figsize=(3, 2))

ddf[["k2_correction", "spin_orbit"]].plot(ax=ax, ls="--", marker="o", lw=1, ms=3)
ax.set_ylabel("$F(q^2)$")
ax.legend(loc="best", fontsize=4, frameon=False)

ax.set_xlabel("$q$ [fm$^{-1}$]")
sns.despine()
plt.show()

In [ ]:
fig.savefig("one-nucleon-corrections.pdf", bbox_inches="tight")